In [1]:
#import libraries
from pymongo import MongoClient
import pandas as pd

In [164]:
def connect_mongo():
    client = MongoClient ('localhost', 27017)
    db = client['companies_clean']
    return db

# Query to MongoDB to get the companies within a radius
def geoquery(lat, long, radius=800):
    db = connect_mongo()
    loc = {"lng": long, "lat": lat}

    result = pd.DataFrame(db.comps.find({
        "coord": {
         "$near": {
           "$geometry": {
              "type": "Point" ,
              "coordinates": [ loc["lng"] , loc["lat"] ]
           },
           "$maxDistance": radius, # In meters 
         }
        }}))
    return result



def add_money_emp():
    db = connect_mongo()
    all_comps = pd.DataFrame(db.comps.find())
    all_comps = all_comps.drop(columns='_id')
    #all_comps = all_comps.loc[all_comps.astype(str).drop_duplicates().index]

    money = []
    employees = []
    for i in range(len(all_comps)):#
        comp = geoquery(all_comps['latitude'][i],all_comps['longitude'][i]).sum(axis = 0, skipna = True)
        try:
            money.append(comp['total_money_raised'])
        except:
            money.append(None)
        try:
            employees.append(comp['number_of_employees'])
        except:
            employees.append(None)
            
    all_comps['employees_neighborhood'] = employees      
    all_comps['money_neighborhood'] = money
    return all_comps

def get_neighborhood(companies, sortby, r):
    companies = companies.sort_values(by=sortby, ascending=False)
    companies = companies.reset_index(drop=True)
    coords = {'lat': companies['latitude'][0],
                'lng': companies['longitude'][0]}
    return geoquery(coords['lat'], coords['lng'], r)

In [165]:
companies = add_money_emp()
companies['$/emp_neighborhood'] = companies['money_neighborhood']/companies['employees_neighborhood']
money_priority = get_neighborhood(companies, 'money_neighborhood', 5000)
people_priority = get_neighborhood(companies, 'employees_neighborhood', 5000)
mpr_priority = get_neighborhood(companies, '$/emp_neighborhood', 5000)


In [168]:
mpr_priority.head()

,_id,category_code,coord,founded_year,latitude,longitude,name,number_of_employees,raised_amount,raised_currency_code,total_money_raised
0,5cd5942a666d28a915fcd954,web,"[72.8545019, 19.0627565]",2007,19.062756,72.854502,Adhysteria,0.0,10000.0,USD,10000.0
